In [1]:
import os 
%pwd


'e:\\TextSummarization\\research'

In [2]:
os.chdir('../')
%pwd

'e:\\TextSummarization'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [4]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml,create_directories

In [5]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

In [6]:
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch

from src.TextSummarizer.logging import logger

e:\TextSummarization\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-07-09 00:05:37,563 - datasets - INFO - PyTorch version 2.7.1 available.


In [ ]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def train(self):
        device='cuda' if torch.cuda.is_available() else 'cpu'
        print(device)
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_t5=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)

        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_t5)

        #loading data
        dataset_samsum_pt=load_from_disk(self.config.data_path)

        logger.info("CPU mode detected. Using a small subset of data for a quick test run.")
        train_dataset_small = dataset_samsum_pt['train'].select(range(200))
        eval_dataset_small = dataset_samsum_pt['validation'].select(range(50))

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=int(self.config.num_train_epochs), 
            warmup_steps=int(self.config.warmup_steps),
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            logging_steps=int(self.config.logging_steps),
            eval_steps=int(self.config.eval_steps),
            save_steps=int(self.config.save_steps),
            gradient_accumulation_steps=int(self.config.gradient_accumulation_steps) 
        )

        trainer=Trainer(
            model=model_t5,args=training_args,
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset_small,
            eval_dataset=eval_dataset_small

        )


        trainer.train()

        model_t5.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum-model'))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))



: 

In [ ]:
try:
    config=ConfigManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.error(e)
    raise e

2025-07-09 00:05:38,115 - TextSummarizerLogger - INFO - YAML file config\config.yaml loaded successfully.
2025-07-09 00:05:38,118 - TextSummarizerLogger - INFO - YAML file params\params.yaml loaded successfully.
2025-07-09 00:05:38,120 - TextSummarizerLogger - INFO - Created Directory at : artifacts
2025-07-09 00:05:38,121 - TextSummarizerLogger - INFO - Created Directory at : artifacts/model_trainer
cpu


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2025-07-09 00:05:54,976 - TextSummarizerLogger - INFO - CPU mode detected. Using a small subset of data for a quick test run.


e:\TextSummarization\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
